<a href="https://colab.research.google.com/github/Mekondjo-EDSA/in2deep/blob/development/MA_starfish_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import os
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -Uqq fastbook

In [ ]:
import fastbook
from fastbook import *
from fastai.vision.all import *
import torch

In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

vid0_path = "/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0"
vid1_path = "/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_1"
vid2_path = "/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_2"
vid_paths = [vid0_path, vid1_path, vid2_path]

vid0_ls = [os.path.join(vid0_path,f) for f in os.listdir(vid0_path)]
vid0_ls = sorted(vid0_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid1_ls = [os.path.join(vid1_path,f) for f in os.listdir(vid1_path)]
vid1_ls = sorted(vid1_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid2_ls = [os.path.join(vid2_path,f) for f in os.listdir(vid2_path)]
vid2_ls = sorted(vid2_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))
files_ls = [vid0_ls, vid1_ls, vid2_ls]

train_df = pd.read_csv("/content/drive/MyDrive/tensorflow-great-barrier-reef/train.csv",
                       sep = r',', skipinitialspace = True)

In [ ]:
print(vid0_ls[:3])
print("\n",files_ls[0][:3])

['/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/0.jpg', '/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/1.jpg', '/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/2.jpg']

 ['/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/0.jpg', '/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/1.jpg', '/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/2.jpg']


In [ ]:
len(files_ls[0])

6708

In [ ]:
train_df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [ ]:
def get_oldpath(x):
    if x.video_id == 0:
        path = os.path.join(vid0_path,str(x.video_frame)+".jpg")
    elif x.video_id == 1:
        path = os.path.join(vid1_path,str(x.video_frame)+".jpg")
    else:
        path = os.path.join(vid2_path,str(x.video_frame)+".jpg")
        
    return path

def get_newpath(x):
    filename = f"{x.video_id}_{x.video_frame}.jpg"
    return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [ ]:
import ast
from tqdm import tqdm
import shutil
train_df = train_df[train_df.annotations != "[]"]
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

os.makedirs("./dataset",exist_ok=True)

for i in tqdm(range(len(train_df))):
    src = train_df.iloc[i]["filepath"]
    dst = train_df.iloc[i]["newpath"]
    shutil.copy(src,dst)
    
train_df.head(3)

100%|██████████| 4919/4919 [02:50<00:00, 28.92it/s] 


,video_id,sequence,video_frame,sequence_frame,image_id,annotations,filepath,newpath,filename
16,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/16.jpg,./dataset/0_16.jpg,0_16.jpg
17,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/17.jpg,./dataset/0_17.jpg,0_17.jpg
18,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",/content/drive/MyDrive/tensorflow-great-barrier-reef/train_images/video_0/18.jpg,./dataset/0_18.jpg,0_18.jpg


In [ ]:
df = train_df
df = df.explode("annotations")

In [ ]:
df['bbox'] = df['annotations'].map(lambda x: list(x.values()))

In [ ]:
df["width"] = [WIDTH]*len(df)
df["height"] = [HEIGHT]*len(df)
df["label"] = ["starfish"]*len(df)

df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)

df.loc[df["xmax"] > 1280, "xmax"] = 1280
df.loc[df["ymax"] > 720, "ymax"] = 720

df = df.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations","filepath","newpath"], axis=1)
df = df.reset_index(drop=True)
df.head(5)

,filename,bbox,width,height,label,xmin,ymin,xmax,ymax
0,0_16.jpg,"[559, 213, 50, 32]",1280,720,starfish,559,213,609,245
1,0_17.jpg,"[558, 213, 50, 32]",1280,720,starfish,558,213,608,245
2,0_18.jpg,"[557, 213, 50, 32]",1280,720,starfish,557,213,607,245
3,0_19.jpg,"[556, 214, 50, 32]",1280,720,starfish,556,214,606,246
4,0_20.jpg,"[555, 214, 50, 32]",1280,720,starfish,555,214,605,246


In [ ]:
root_dir = './'
train_dir = os.path.join(root_dir,'dataset')

In [ ]:
df['image_name'] = train_dir+'/'+df['filename']

In [ ]:
df.head()

,filename,bbox,width,height,label,xmin,ymin,xmax,ymax,image_name
0,0_16.jpg,"[559, 213, 50, 32]",1280,720,starfish,559,213,609,245,./dataset/0_16.jpg
1,0_17.jpg,"[558, 213, 50, 32]",1280,720,starfish,558,213,608,245,./dataset/0_17.jpg
2,0_18.jpg,"[557, 213, 50, 32]",1280,720,starfish,557,213,607,245,./dataset/0_18.jpg
3,0_19.jpg,"[556, 214, 50, 32]",1280,720,starfish,556,214,606,246,./dataset/0_19.jpg
4,0_20.jpg,"[555, 214, 50, 32]",1280,720,starfish,555,214,605,246,./dataset/0_20.jpg


In [ ]:
df['xmin'] = df['xmin'].astype(int)
df['ymin'] = df['ymin'].astype(int)
df['xmax'] = df['xmax'].astype(int)
df['ymax'] = df['ymax'].astype(int)

In [ ]:
df.drop(columns=['filename', 'width','height'], axis=1, inplace=True)

In [ ]:
column_names = ["image_name","bbox" ,"xmin", "ymin", "xmax", "ymax", "label"]
df = df.reindex(columns=column_names)

In [ ]:
df.to_csv('mainframe.csv',index=False)

In [ ]:
df.head()

,image_name,bbox,xmin,ymin,xmax,ymax,label
0,./dataset/0_16.jpg,"[559, 213, 50, 32]",559,213,609,245,starfish
1,./dataset/0_17.jpg,"[558, 213, 50, 32]",558,213,608,245,starfish
2,./dataset/0_18.jpg,"[557, 213, 50, 32]",557,213,607,245,starfish
3,./dataset/0_19.jpg,"[556, 214, 50, 32]",556,214,606,246,starfish
4,./dataset/0_20.jpg,"[555, 214, 50, 32]",555,214,605,246,starfish


In [ ]:
!pip install -Uqq fastbook

In [ ]:

import fastbook
from fastbook import *
from fastai.vision import *
from fastai.callback.tracker import SaveModelCallback

In [ ]:
fastbook.setup_book()

OSError: ignored

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import os
from fastai.callback import *
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.image as immg
from sklearn.model_selection import StratifiedKFold,KFold

In [ ]:
df.head(5)

In [ ]:
path = Path('/content/dataset')

In [ ]:
img_files = get_image_files(path)
img = PILImage.create(img_files[100])
img

In [ ]:
img.to_thumb(180)

In [ ]:
path.ls()

In [ ]:
img_size = 128
augmentations = [
        Rotate(10, p=0.4, mode='bilinear'), 
        Brightness(max_lighting=0.3,p=0.5),
        Contrast(max_lighting=0.4, p=0.5),
        RandomErasing(p=0.3, sl=0.0, sh=0.2, min_aspect=0.3, max_count=1),
        Flip(p=0.5),
        Zoom(max_zoom=1,p=0.5),
        RandomResizedCrop(img_size)
    ]

In [ ]:
dblock = DataBlock()

In [ ]:
dsets = dblock.datasets(df)


In [ ]:
dsets[0:5]

In [ ]:
dsets.train[0]

In [ ]:
def get_y(r):
  return lambda r: r ['labels']

In [ ]:
img_size = 128
augmentations = [
        Rotate(10, p=0.4, mode='bilinear'), 
        Brightness(max_lighting=0.3,p=0.5),
        Contrast(max_lighting=0.4, p=0.5),
        RandomErasing(p=0.3, sl=0.0, sh=0.2, min_aspect=0.3, max_count=1),
        Flip(p=0.5),
        Zoom(max_zoom=1,p=0.5),
        RandomResizedCrop(img_size)
    ]
dls = ImageDataLoaders.from_df(df=df,
                               path=".",
                               label_col = 6,
                               valid_pct = 0.2, 
                               bs = 32, 
                               device=device, 
                               num_workers=0,
                               batch_tfms=augmentations,
                               item_tfms=Resize(img_size))
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, 
                    resnet34, 
                    metrics= error_rate)
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1)

# -------------------------------------------------------------------------------

In [ ]:
train_df = pd.read_csv('/content/mainframe.csv')

In [ ]:
train_df.head()

In [ ]:
coco_source = untar_data(URLs.COCO_TINY)
images, lbl_bbox = get_annotations(coco_source/'train.json')
img2bbox = dict(zip(images, lbl_bbox))

In [ ]:
images[0],lbl_bbox[0]

In [ ]:
def get_lbl_img(train):
    star2bbox = {}
    train['label'] = 'starfish'
    grp = train.image_name.unique()
    tr_gr = train.groupby(['image_name'])
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(grp))):
        name = str(grp[i])
        bbox = []
        temp_b = []
        temp = tr_gr.get_group(grp[i])
        tt = temp.loc[:,'bbox'].values
        for j in range(len(temp)):
            t = tt[j][1:-1].split(',')
            t = [float(x) for x in t]  # x,y, width, height
            # Currently our coordinates are x,y,w,h and we want x1,y1,x2,y2
            # To convert it, we need to add our width and height to the respective x and y.
            t[2],t[3] = t[0]+t[2],t[1]+t[3]  
            # To achieve x2,y2 those we simply add width to x and height to y :
            # x2 = x + w and y2 = y + h
            t1 = [t[1],t[0],t[3],t[2]]   # inverse in fromat w,h for fastai
            temp_b.append(t1)
        bbox.append(temp_b)
        bbox.append(['starfish']*len(temp))
        star2bbox[name] = bbox
    return star2bbox

In [ ]:
star2bbox = get_lbl_img(train_df)

In [ ]:
a = star2bbox['./dataset/0_4777.jpg']; a

In [ ]:
get_y_func = lambda x: star2bbox[Path(x).label] 

In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/tensorflow-great-barrier-reef/example_sample_submission.csv')

In [ ]:
sample.head()